**Runner Registration per Week**

In [5]:
SELECT COUNT(runner_id) runner_registration, DATEPART(WEEK, registration_date) registration_week
FROM runners
GROUP BY DATEPART(WEEK, registration_date)
ORDER BY registration_week

(3 rows affected)

Total execution time: 00:00:00.034

runner_registration,registration_week
1,1
2,2
1,3


**Data Cleaning**

In [6]:
BEGIN TRANSACTION

UPDATE runner_orders
SET duration = REPLACE(duration, 'minutes', '')

UPDATE runner_orders
SET duration = REPLACE(duration, 'minute', '')

UPDATE runner_orders
SET duration = REPLACE(duration, 'mins', '')

ALTER TABLE runner_orders
ALTER COLUMN duration INT

ALTER TABLE runner_orders
ALTER COLUMN pickup_time datetime

COMMIT

(10 rows affected)

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.089

**<mark>Average</mark> Arrival Time (Minutes) to HQ**

In [7]:
SELECT runner_id, ABS(AVG(DATEPART(MINUTE, order_time) - DATEPART(MINUTE, pickup_time))) avg_time
FROM runner_orders r JOIN customer_orders c
ON r.order_id = c.order_id
GROUP BY runner_id

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.042

runner_id,avg_time
1,4
2,12
3,10


**Pizzas Made per Preparation Time (Minutes)**

In [8]:
WITH prep_time AS (
SELECT COUNT(*) pizza_count, 
DATEDIFF(MINUTE, order_time, pickup_time) time
FROM customer_orders c JOIN runner_orders r
ON c.order_id = r.order_id
WHERE pickup_time IS NOT NULL
GROUP BY order_time, pickup_time 
)
SELECT DISTINCT pizza_count, time
FROM prep_time
ORDER BY time

(5 rows affected)

Total execution time: 00:00:00.030

pizza_count,time
1,10
2,16
1,21
2,21
3,30


**<mark>Average</mark> <mark>D</mark>istance (km) Travelle<mark>d</mark> per Customer**

In [9]:
BEGIN TRANSACTION
UPDATE runner_orders
SET distance = REPLACE(distance, 'km', '')

ALTER TABLE runner_orders
ALTER COLUMN distance FLOAT
COMMIT

(10 rows affected)

Total execution time: 00:00:00.015

In [10]:
SELECT runner_id, customer_id, AVG(CAST(distance as INT)) avg_distance
FROM runner_orders r JOIN customer_orders c
ON r.order_id = c.order_id
WHERE cancellation IS NULL
GROUP BY runner_id, customer_id

(7 rows affected)

Total execution time: 00:00:00.042

runner_id,customer_id,avg_distance
1,101,20
1,102,13
2,102,23
2,103,23
1,104,10
3,104,10
2,105,25


**Shortest - Longest Delivery Times**

In [11]:
SELECT MAX(duration ) - MIN(duration) 'difference between longest_and shortest orders'
FROM runner_orders r JOIN customer_orders c
ON r.order_id = c.order_id

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.017

difference between longest_and shortest orders
30


**Average Speed per Delivery**

In [12]:
SELECT runner_id, order_id AS delivery, AVG(ROUND(distance/duration, 2)) avg_speed
FROM runner_orders
WHERE cancellation IS NULL
GROUP BY runner_id, order_id

(8 rows affected)

Total execution time: 00:00:00.035

runner_id,delivery,avg_speed
1,1,0.63
1,2,0.74
1,3,0.67
2,4,0.58
3,5,0.67
2,7,1
2,8,1.56
1,10,1


**Successful Delivery % per Runner**

In [39]:
SELECT runner_id,
ROUND(CAST(SUM(
CASE 
    WHEN cancellation IS NOT NULL THEN 0
    ELSE 1    
END) AS FLOAT) / COUNT(*), 2) * 100 as '%'
FROM runner_orders
GROUP BY runner_id

(3 rows affected)

Total execution time: 00:00:00.019

runner_id,%
1,100
2,75
3,50
